In [4]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"

import torch

# 檢查結果
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")


# 選擇 GPU 編號（根據 CUDA_VISIBLE_DEVICES 的排序，如果沒設定，就是實體順序）
selected_gpu = 0
device = torch.device(f"cuda:{selected_gpu}" if torch.cuda.is_available() else "cpu")

# 顯示目前選到的 GPU 型號
if device.type == 'cuda':
    gpu_name = torch.cuda.get_device_name(selected_gpu)
    print(f"Using GPU {selected_gpu}: {gpu_name}")
else:
    print("Using CPU")

dtype = torch.bfloat16 if torch.cuda.get_device_capability()[0] >= 8 else torch.float16

GPU 0: NVIDIA RTX A5000
GPU 1: NVIDIA RTX A6000
GPU 2: NVIDIA RTX A5000
Using GPU 0: NVIDIA RTX A5000


In [5]:
import sys
sys.path.append("/home/lyc/research/vggt")

In [6]:
import glob
import open3d as o3d
import numpy as np
from plyfile import PlyData, PlyElement
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from vggt.models.vggt import VGGT
from vggt.utils.load_fn import load_and_preprocess_images
from vggt.utils.pose_enc import pose_encoding_to_extri_intri



In [7]:
# 初始化模型（並自動下載權重）
model = VGGT.from_pretrained("facebook/VGGT-1B").to(device)

# 設定資料夾名稱與圖片路徑
image_folder = "data/Images/1"
image_paths = sorted(glob.glob(os.path.join(image_folder, "*")))

# 預處理
images = load_and_preprocess_images(image_paths).to(device)

# 推論
with torch.no_grad():
    with torch.cuda.amp.autocast(dtype=dtype):
        predictions = model(images)


# 還原 extrinsics 和 intrinsics
extri, intri = pose_encoding_to_extri_intri(predictions["pose_enc"], images.shape[-2:])

# 將 extrinsics 和 intrinsics 加入 predictions
predictions["extri"] = extri
predictions["intri"] = intri

# 儲存成 .pt，檔名與資料夾名稱相同
output_filename = f"{os.path.basename(image_folder)}.pt"
torch.save(predictions, output_filename)
print(f"✅ 已儲存至 {output_filename}")

✅ 已儲存至 1.pt


In [8]:
for key, value in predictions.items():
    if isinstance(value, torch.Tensor):
        print(f"{key}: {value.shape}")
    else:
        print(f"{key}: {type(value)}")


pose_enc: torch.Size([1, 17, 9])
depth: torch.Size([1, 17, 392, 518, 1])
depth_conf: torch.Size([1, 17, 392, 518])
world_points: torch.Size([1, 17, 392, 518, 3])
world_points_conf: torch.Size([1, 17, 392, 518])
images: torch.Size([1, 17, 3, 392, 518])
extri: torch.Size([1, 17, 3, 4])
intri: torch.Size([1, 17, 3, 3])
